## Чат-Бот Психологической поддержки

##### Техническое задание: программа-чат-бота, которая общается с пользователем в виде текста: выводит текст, считывает ответы пользователя и в зависимости от этих ответом дает советы.

Критерии выполнения:
> - практическая применимость
> - 4 окончания
> - сложное условие
> - путь минимум 10 шагов
> - повторяющаяся часть программы
> - вывод текста
> - понятный интерфейс
> - комментарии (не менее 30)
> - функции
> - запись в файл


##### Описание:
Бот Жора - https://t.me/psycologicalsupportbot
Что умеет этот бот? 
Жора - это твой виртуальный психолог и помощник в любых жизненных ситуациях.
Жора поможет справиться со стрессом, низкой самооценкой, расставанием, потерей смысла, отсутствием продуктивности и мотивации. 

**Важно: это не медицинский инструмент и бот не работает с кризисными ситуациями.Конфиденциальность гарантируется.**

Для создания бота и работы с телеграм была использована библиотека *aiogram*,
 импортированы модули для работы с клавиатурой, переменным окружением, работы и запуска бота
 импортирован модуль *difflib* для сравнения последовательностей и поиска наиболее подходящих ответов на вопросы пользователя
 последовательность шагов определяется машиной состояний:
 
> - **1 шаг**: запуск и получение input от пользователя
> - **2 шаг**: открытие основного меню и переход в диалог с ботом
> - **3 шаг**: чат с ботом, запись отсутствующего ответа в словарь (повторяющаяся часть программы до state.finish())
> - **4 шаг**: завершение диалога дает возможность получить совет от бота 

Запись ответов пользователя происходит в файл "answers.json" в качестве одного из путей пользователя.

Запуск бота осуществляется по нажатию кнопки "start" или команде /start
Запуск локально: http://localhost:****/notebooks/main.ipynb

##### Вариант беседы с ботом, состоящий из 10 шагов (записан в answers.json):

![скриншот_json](json.png)

In [ ]:
#импортирую необходимые модули
import os
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv

from aiogram import Bot, types, Dispatcher
from aiogram.utils import executor
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from aiogram.dispatcher import FSMContext
from aiogram.dispatcher.filters.state import State, StatesGroup

from aiogram.types.web_app_info import WebAppInfo
from aiogram.types import InlineKeyboardButton, InlineKeyboardMarkup, ReplyKeyboardMarkup

import json
from difflib import get_close_matches

#Основное меню бота, прописываю инлайн кнопки

b2 = InlineKeyboardButton('Обсудить проблему', callback_data='button2')
b4 = InlineKeyboardButton('Совет на день', callback_data='button4')
back_to_menu = InlineKeyboardButton('Вернуться в главное меню', callback_data='back')
website = InlineKeyboardButton('Сервис по подбору психологов', web_app=WebAppInfo(url = 'https://yasno.live/'))
give_advice = InlineKeyboardButton('Получить совет', callback_data='advice')

keyb_client = InlineKeyboardMarkup().add(b2).insert(b4).add(website) #создаем меню и передаем в функцию
keyb_client01 = InlineKeyboardMarkup().add(give_advice)

#путь к файлу, в котором записаны ответы и вопросы пользователей
file_path = '/Users/alenaagafonova/PycharmProjects/TGbot/Psychologi/AleksandraAgafonova/answers.json'

#четыре функции на открытие, поиск и сохранение ответов
def load_answers(file_path: str) -> dict:
    with open(file_path, 'r') as file:
        data: dict = json.load(file)
    return data

def save_answers(file_path: str, data: dict):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=2, ensure_ascii=False)
        file.close()

#функция поиска подходящих ответов
def find_best_match(user_question: str, questions: list[str]) -> str| None:
    matches: list = get_close_matches(user_question, questions, n=1, cutoff=0.6)#выставляем прааметр ближе к единице для достоверности результатов
    return matches[0] if matches else None

#получаем ответ бота
def get_answers_for_questions(question: str, answers: dict) -> str | None:
    for q in answers['questions']:
        if q['question'] == question:
            return q['answer']


#Создаю объекты бота и диспетчера
load_dotenv()
bot = Bot(os.getenv('TOKEN'))
DB_URL = os.getenv('POSTRGESS_URI')
dp = Dispatcher(bot=bot, storage=MemoryStorage())
test = os.getenv('test')

#прописываю классы машины состояний
class FSMAdmin(StatesGroup):
    message =State()

class Start(StatesGroup):
    age = State()
    next_step = State()
    chat_bot = State()
    payment = State()

photo = open('/Users/alenaagafonova/Desktop/НИС/photo_2024-02-24_12-09-46.jpg', 'rb')

#декораторы для запуска бота с приветсвием и запросом возраста
##декоратор обрабатывает ответ пользователя и возвращает в следующем декораторе
@dp.message_handler(commands=['start'])
async def start(msg: types.Message, state: FSMContext):
    await bot.send_message(msg.from_user.id,
                           text="Для эффективной работы с ботом, пожалуйста, напиши мне в ответ свой возраст в формате: 18 лет/ 22 года.")
    await Start.age.set()#перехожу в режим ожидания ответа
    
##декоратор принимает значение введенное пользователем и выводит в ответе
@dp.message_handler(state=Start.age)
async def start(msg: types.Message, state: FSMContext):
    await state.update_data(user_age=msg.text)
    data = await state.get_data()#получаю и записываю ответ в FSM
    with open('/Users/alenaagafonova/Desktop/НИС/photo_2024-02-24_12-09-46.jpg', 'rb') as photo:
        await bot.send_photo(msg.from_user.id, caption = f"Привет, {msg.from_user.full_name}\! Я уже знаю, что тебе {data['user_age']} \. Давай знакомиться дальше\."
                                                         f"\n\nЯ Жора, твой виртуальный психолог и помощник в любых жизненных ситуациях\.\n\nПожалуйста, выбери интересующий раздел\.",
                         parse_mode='MarkdownV2',photo=photo,
                         reply_markup=keyb_client)
        await Start.next_step.set()
        
#декоратор обработки основных команд для callback запрсов и работы с инлайн кнопками
@dp.callback_query_handler(lambda c: True, state=Start.next_step)
async def start_chat(callback_query: types.CallbackQuery): #объявляю функцию, в которую передаю апдейт типа CallbackQuery
    if callback_query.data == 'button2':#переход в режим диалога с ботом
        await callback_query.bot.send_message(callback_query.from_user.id,
            'Чем я могу тебе помочь? \n\n<i>Напиши свой вопрос и я постарюсь на него ответить</i>', parse_mode='HTML')
        await Start.chat_bot.set()

    if callback_query.data == 'website': #открытие WebApp handler
        await callback_query.message.edit_caption(
            'Ты перешел на страницу сайта Ясно.',
            reply_markup=keyb_client) #вариант окончания работы

#условие оплаты обязательное для продолжения работы с ботом
@dp.callback_query_handler(lambda c: True, state=None)
async def payment(callback_query: types.CallbackQuery):
    if callback_query.data == 'button4':
        await bot.send_invoice(chat_id=callback_query.from_user.id, title='Оплата подписки на бот', description='Активация подписки на бот на один месяц',
                           payload='test-invoice-payload', provider_token=test, currency='RUB',
  
                               start_parameter='one-month-subscription', prices=[{'label': 'RUB', 'amount': 100000}])
#проверяю получение оплаты
@dp.pre_checkout_query_handler(lambda query:True)
async def process_pre_checkout_query(query: types.PreCheckoutQuery):
    await bot.answer_pre_checkout_query(pre_checkout_query_id=query.id, ok = True)
    await bot.send_message(chat_id=query.from_user.id, text = "Спасибо за покупку. Теперь ты можешь получать советы от бота.", reply_markup=keyb_client01) #вариант окончания работы

async def payment_success(callback_query: types.CallbackQuery):
    if callback_query.data == 'advice':
        await callback_query.bot.send_message(callback_query.from_user.id,
                                              'Первый совет: награди себя, не забывай себя радовать, дари себе подарки и приятности. Это поможет чувстовть себя лучше и настроит на новые достижения.',
                                              parse_mode='HTML')

#самый длинный путь в 10 шагов и более обернут в последующие два декоратора,
#без выполнения этого условия невозможно перейти к оплате (хэндлер обработки выше)
@dp.message_handler(state=Start.chat_bot)
async def chat_bot(msg: types.Message, state: FSMContext):
    answers: dict = load_answers("/Users/alenaagafonova/PycharmProjects/TGbot/Psychologi/AleksandraAgafonova/answers.json") #открываю словарь
    user_input: str = msg.text #собираю сообщение
    async with state.proxy() as data: #сохраняю данные в MemoryStorage, чтобы передать в следующий хэндлер
        data['user_input'] = user_input

    best_match: str | None = find_best_match(user_input, [q["question"] for q in answers['questions']]) #ищу наиболее подходящие вопросы в словаре

    if best_match:
        answer: str = get_answers_for_questions(best_match, answers)
        await bot.send_message(text = f'{answer}', chat_id=msg.chat.id) #получаю и отправляю ответ пользователю
    else:
        await bot.send_message(text = "Я не понимаю тебя. Может, научишь меня? Если напишешь свой ответ, я сохраню его. Если захочешь закончить диалог просто напиши \"пропустить\" или \"завершить сеанс\", чтобы закончить диалог.", chat_id=msg.chat.id)
        await FSMAdmin.next() #при отстусвие совпадений перехожу в следующий хэндлер

@dp.message_handler(state=FSMAdmin.message)
async def chat_bot_add(msg: types.Message, state: FSMContext):
    async with state.proxy() as data:
        user_input = data['user_input']
    answers: dict = load_answers("/Users/alenaagafonova/PycharmProjects/TGbot/Psychologi/AleksandraAgafonova/answers.json") #открываю словарь
    new_answer: str = msg.text #принимаю на запись новое сообщение

    if new_answer.lower() == "пропустить" or new_answer.lower() == "завершить сеанс": #сложное условие с использованием or
        await bot.send_message(text='Был рад тебе помочь! Увидимся позже.', chat_id=msg.chat.id) #окончание работы бота
    else:
        answers['questions'].append({'question': user_input, "answer": new_answer})
        save_answers('answers.json', answers) #записываем ответ пользователя в словарь
        await bot.send_message(text='Спасибо! Я стал немного умнее.', chat_id=msg.chat.id) #окончание работы бота
    await state.finish() #завершаем состояние

if __name__ == '__main__':
    executor.start_polling(dp, skip_updates=True)

Updates were skipped successfully.
